In [1]:
import threading
import multiprocessing
import numpy as np
import tensorflow as tf
from AtariEnvironment import *
from ACERNetwork import * 
from ReplayBuffer import *
from Worker import * 
from Trainer import train,train_acer, test
import gym as gym
from time import sleep
from time import time
from utils import preprocess

network_t = "cnn"
GAME_NAME="Breakout-v0"
#GAME_NAME = "LunarLander-v2"
TB_DIR = "saves/breakout/test4replay8_e4/"
env=Atari_Environment(gym.make(GAME_NAME))
#env = gym.make(GAME_NAME)
state= env.reset()
training_finished = False
LEARNING_RATE = 1e-4
print(env.action_space)

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Discrete(4)


/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [2]:
from tensorflow import Session, ConfigProto
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
config = ConfigProto()
config.gpu_options.allow_growth=True
Session(config=config).close()

In [3]:
session = tf.Session()
tf.reset_default_graph()

with tf.Session() as sess:
    with tf.device('/cpu:0'):
        G_Agent=ACERNetwork(sess,env,"global",network_type=network_t)
        agent = ACERNetwork(sess,env,str(0),G_Agent, network_type=network_t)
        sess.run(tf.global_variables_initializer())
        print(agent.get_pi(state))
        print(G_Agent.get_pi(state))
        agent.update_target()
        print(agent.get_pi(state))
        print(G_Agent.get_pi(state))


Tensor("net_global/shared_policy_net/conv_layer_1/Relu:0", shape=(?, 20, 20, 32), dtype=float32, device=/device:CPU:0)
Tensor("net_global/shared_policy_net/conv_layer_2/Relu:0", shape=(?, 9, 9, 64), dtype=float32, device=/device:CPU:0)
Tensor("net_global/shared_policy_net/conv_layer_3/Relu:0", shape=(?, 7, 7, 64), dtype=float32, device=/device:CPU:0)
Tensor("net_0/shared_policy_net/conv_layer_1/Relu:0", shape=(?, 20, 20, 32), dtype=float32, device=/device:CPU:0)
Tensor("net_0/shared_policy_net/conv_layer_2/Relu:0", shape=(?, 9, 9, 64), dtype=float32, device=/device:CPU:0)
Tensor("net_0/shared_policy_net/conv_layer_3/Relu:0", shape=(?, 7, 7, 64), dtype=float32, device=/device:CPU:0)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


[[0.2606373  0.23990695 0.2443359  0.25511983]]
[[0.25480744 0.24302219 0.2540271  0.24814327]]
[[0.25480744 0.24302219 0.2540271  0.24814327]]
[[0.25480744 0.24302219 0.2540271  0.24814327]]


In [4]:
def start_method(game_name=GAME_NAME, num_agents = 64, restore = False, save_path=TB_DIR, replay_ratio=0):
        
    envs=[]
    agents=[]
    workers=[]
    processes=[]
    
    for _ in range(num_agents):
        #a_env = gym.make(game_name)
        #a_env = Custom_Environment(gym.make(game_name))
        a_env = Atari_Environment(gym.make(game_name), clipreward=True)
        envs.append(a_env)
       

    with tf.Session() as sess:
        with tf.device('/cpu:0'):
            coord = tf.train.Coordinator()
            G_Agent=ACERNetwork(sess,envs[0],"global",network_type=network_t,lr=LEARNING_RATE,decay=0.99)
            for agent_num in range(num_agents):
                agents.append(ACERNetwork(sess,envs[0],str(agent_num),G_Agent,network_type=network_t))
            saver = tf.train.Saver(max_to_keep=5)
            

            if restore:
                ckpt = tf.train.get_checkpoint_state(save_path+"checkpoints")
                saver.restore(sess,ckpt.model_checkpoint_path)
            else:
                sess.run(tf.global_variables_initializer())
            # Create a process for each worker
            for thread_id in range(num_agents):
                if thread_id == 0:
                    workers.append(Worker(agents[thread_id],envs[thread_id],sess,"eval", 20, 0.99, 200000, replay_ratio))
                    processes.append(threading.Thread(target=workers[thread_id].work_and_eval_acer, args=(saver,TB_DIR)))
                else:
                    workers.append(Worker(agents[thread_id],envs[thread_id],sess,str(thread_id), 20, 0.99, 200000, replay_ratio))
                    processes.append(threading.Thread(target=workers[thread_id].work_acer))
            # Start all the processes
            for p in processes:
                p.daemon = True
                p.start()
                
            # Join the processes, so we get this thread back.
            coord.join(processes)


In [ ]:
tf.reset_default_graph()
start_method(replay_ratio=4)

Tensor("net_global/shared_policy_net/conv_layer_1/Relu:0", shape=(?, 20, 20, 32), dtype=float32, device=/device:CPU:0)
Tensor("net_global/shared_policy_net/conv_layer_2/Relu:0", shape=(?, 9, 9, 64), dtype=float32, device=/device:CPU:0)
Tensor("net_global/shared_policy_net/conv_layer_3/Relu:0", shape=(?, 7, 7, 64), dtype=float32, device=/device:CPU:0)
Tensor("net_0/shared_policy_net/conv_layer_1/Relu:0", shape=(?, 20, 20, 32), dtype=float32, device=/device:CPU:0)
Tensor("net_0/shared_policy_net/conv_layer_2/Relu:0", shape=(?, 9, 9, 64), dtype=float32, device=/device:CPU:0)
Tensor("net_0/shared_policy_net/conv_layer_3/Relu:0", shape=(?, 7, 7, 64), dtype=float32, device=/device:CPU:0)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Tensor("net_1/shared_policy_net/conv_layer_1/Relu:0", shape=(?, 20, 20, 32), dtype=float32, device=/device:CPU:0)
Tensor("net_1/shared_policy_net/conv_layer_2/Relu:0", shape=(?, 9, 9, 64), dtype=float32, device=/device:CPU:0)
Tensor("net_1/shared_policy_net/conv_layer_3/Relu:0", shape=(?, 7, 7, 64), dtype=float32, device=/device:CPU:0)
Tensor("net_2/shared_policy_net/conv_layer_1/Relu:0", shape=(?, 20, 20, 32), dtype=float32, device=/device:CPU:0)
Tensor("net_2/shared_policy_net/conv_layer_2/Relu:0", shape=(?, 9, 9, 64), dtype=float32, device=/device:CPU:0)
Tensor("net_2/shared_policy_net/conv_layer_3/Relu:0", shape=(?, 7, 7, 64), dtype=float32, device=/device:CPU:0)
Tensor("net_3/shared_policy_net/conv_layer_1/Relu:0", shape=(?, 20, 20, 32), dtype=float32, device=/device:CPU:0)
Tensor("net_3/shared_policy_net/conv_layer_2/Relu:0", shape=(?, 9, 9, 64), dtype=float32, device=/device:CPU:0)
Tensor("net_3/shared_policy_net/conv_layer_3/Relu:0", shape=(?, 7, 7, 64), dtype=float32, device=/

Tensor("net_26/shared_policy_net/conv_layer_1/Relu:0", shape=(?, 20, 20, 32), dtype=float32, device=/device:CPU:0)
Tensor("net_26/shared_policy_net/conv_layer_2/Relu:0", shape=(?, 9, 9, 64), dtype=float32, device=/device:CPU:0)
Tensor("net_26/shared_policy_net/conv_layer_3/Relu:0", shape=(?, 7, 7, 64), dtype=float32, device=/device:CPU:0)
Tensor("net_27/shared_policy_net/conv_layer_1/Relu:0", shape=(?, 20, 20, 32), dtype=float32, device=/device:CPU:0)
Tensor("net_27/shared_policy_net/conv_layer_2/Relu:0", shape=(?, 9, 9, 64), dtype=float32, device=/device:CPU:0)
Tensor("net_27/shared_policy_net/conv_layer_3/Relu:0", shape=(?, 7, 7, 64), dtype=float32, device=/device:CPU:0)
Tensor("net_28/shared_policy_net/conv_layer_1/Relu:0", shape=(?, 20, 20, 32), dtype=float32, device=/device:CPU:0)
Tensor("net_28/shared_policy_net/conv_layer_2/Relu:0", shape=(?, 9, 9, 64), dtype=float32, device=/device:CPU:0)
Tensor("net_28/shared_policy_net/conv_layer_3/Relu:0", shape=(?, 7, 7, 64), dtype=float32,

Tensor("net_51/shared_policy_net/conv_layer_1/Relu:0", shape=(?, 20, 20, 32), dtype=float32, device=/device:CPU:0)
Tensor("net_51/shared_policy_net/conv_layer_2/Relu:0", shape=(?, 9, 9, 64), dtype=float32, device=/device:CPU:0)
Tensor("net_51/shared_policy_net/conv_layer_3/Relu:0", shape=(?, 7, 7, 64), dtype=float32, device=/device:CPU:0)
Tensor("net_52/shared_policy_net/conv_layer_1/Relu:0", shape=(?, 20, 20, 32), dtype=float32, device=/device:CPU:0)
Tensor("net_52/shared_policy_net/conv_layer_2/Relu:0", shape=(?, 9, 9, 64), dtype=float32, device=/device:CPU:0)
Tensor("net_52/shared_policy_net/conv_layer_3/Relu:0", shape=(?, 7, 7, 64), dtype=float32, device=/device:CPU:0)
Tensor("net_53/shared_policy_net/conv_layer_1/Relu:0", shape=(?, 20, 20, 32), dtype=float32, device=/device:CPU:0)
Tensor("net_53/shared_policy_net/conv_layer_2/Relu:0", shape=(?, 9, 9, 64), dtype=float32, device=/device:CPU:0)
Tensor("net_53/shared_policy_net/conv_layer_3/Relu:0", shape=(?, 7, 7, 64), dtype=float32,

/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Worker  worker_eval At  1228  Running/Max:  1.2000000000000002 3.0  Frames: 20
pi: [[0.27637666 0.25264046 0.24533105 0.22565179]]
Saving Model
Worker  worker_eval At  10131  Running/Max:  1.8873130988000004 4.0  Frames: 158
pi: [[0.24321936 0.3209772  0.23136625 0.20443718]]
Saving Model
Worker  worker_eval At  20482  Running/Max:  1.9569840475628935 4.0  Frames: 318
pi: [[0.28464028 0.2628582  0.2661269  0.18637456]]
Saving Model
Worker  worker_eval At  30429  Running/Max:  2.340892318672016 4.0  Frames: 471
pi: [[0.20279308 0.31633925 0.32726043 0.15360723]]
Saving Model
Worker  worker_eval At  40443  Running/Max:  2.2686528508166317 4.0  Frames: 627
pi: [[0.22567682 0.29552832 0.3055267  0.17326814]]
Saving Model
Worker  worker_eval At  50382  Running/Max:  2.7970440031495825 6.0  Frames: 780
pi: [[0.22511345 0.25917447 0.25370204 0.26201004]]
Saving Model
Worker  worker_eval At  64152  Running/Max:  2.171410862077818 6.0  Frames: 863
pi: [[0.23383328 0.23198628 0.4323884  0.101791

In [ ]:
TB_DIR = "saves/spaceinv/final_R1/"
tf.reset_default_graph()
start_method(replay_ratio=1)

In [ ]:
def test_method(game_name=GAME_NAME, num_agents = 1, restore = True, save_path=TB_DIR):
        
    envs=[]
    agents=[]
    processes=[]
    
    for _ in range(num_agents):
        #a_env = gym.make(game_name)
        #a_env = Custom_Environment(gym.make(game_name))
        a_env = Atari_Environment(gym.make(game_name), clipreward=False)
        envs.append(a_env)
       


    with tf.Session() as sess:
        with tf.device('/cpu:0'):
            coord = tf.train.Coordinator()
            G_Agent=ACERNetwork(sess,envs[0],"global",network_type=network_t,lr=LEARNING_RATE,decay=0.99)
            for agent_num in range(num_agents):
                agents.append(ACERNetwork(sess,envs[0],str(agent_num),G_Agent,network_type=network_t))
            saver = tf.train.Saver(max_to_keep=2)

            if restore:
                ckpt = tf.train.get_checkpoint_state(save_path+"checkpoints")
                saver.restore(sess,ckpt.model_checkpoint_path)
            else:
                sess.run(tf.global_variables_initializer())
                
            for thread_id in range(num_agents):
                processes.append(threading.Thread(target=test, args=(agents[thread_id],
                envs[thread_id], 10, True)))

            for p in processes:
                p.daemon = True
                p.start()

                
            for p in processes:
                p.join()


In [ ]:
tf.reset_default_graph()
test_method()